### Forward propagation:


```python
Z1 = X.W1 + b1
A1 = ReLU(Z1)  
Z2 = A1.W2 + b2
exp_scores = exp(Z2)  
probs = exp_scores / sum(exp_scores)
```

### Backward propagation:

```python 
delta3 = probs
delta3[range(len(X)), y] -= 1
dW2 = A1.T.dot(delta3)
db2 = sum(delta3)
delta2 = delta3.dot(W2.T) * (A1 > 0)
dW1 = X.T.dot(delta2)
db1 = sum(delta2)

```

Here:

X is the input data matrix of shape (num_samples, input_size), W1 is the weight matrix connecting the input layer to the hidden layer of shape (input_size, hidden_size), b1 is the bias vector for the hidden layer of shape (hidden_size,), A1 is the output of the hidden layer (also known as the hidden representation) of shape (num_samples, hidden_size), W2 is the weight matrix connecting the hidden layer to the output layer of shape (hidden_size, output_size), b2 is the bias vector for the output layer of shape (output_size,), Z2 is the weighted sum of the hidden layer output, exp_scores is the exponential of the output layer weighted sum, probs is the output probability for each class, and y is the true label vector of shape (num_samples,).



## Code

In [2]:
import numpy as np

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        self.params = {}
        self.params['W1'] = np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def forward(self, X):
        W1, b1 = self.params['W1'], self.params['b1']
        W2, b2 = self.params['W2'], self.params['b2']
        z1 = np.dot(X, W1) + b1
        a1 = np.maximum(0, z1) # ReLU activation function
        z2 = np.dot(a1, W2) + b2
        # probs = 1 / (1 + np.exp(-z2)) # Sigmoid activation function
        exp_z = np.exp(z2)
        probs = exp_z / np.sum(exp_z, axis=1, keepdims=True)
        return probs

    def loss(self, X, y):
        probs = self.forward(X)
        correct_logprobs = -np.log(probs[range(len(X)), y])
        data_loss = np.sum(correct_logprobs)
        return 1.0/len(X) * data_loss

    def train(self, X, y, num_epochs, learning_rate=0.1):
        for epoch in range(num_epochs):
            # Forward propagation
            z1 = np.dot(X, self.params['W1']) + self.params['b1']
            a1 = np.maximum(0, z1) # ReLU activation function
            z2 = np.dot(a1, self.params['W2']) + self.params['b2']
            # probs = 1 / (1 + np.exp(-z2)) # Sigmoid activation function
            exp_z = np.exp(z2)
            probs = exp_z / np.sum(exp_z, axis=1, keepdims=True)

            # Backpropagation
            delta3 = probs
            delta3[range(len(X)), y] -= 1
            dW2 = np.dot(a1.T, delta3)
            db2 = np.sum(delta3, axis=0)
            delta2 = np.dot(delta3, self.params['W2'].T) * (a1 > 0) # derivative of ReLU
            dW1 = np.dot(X.T, delta2)
            db1 = np.sum(delta2, axis=0)

            # Update parameters
            self.params['W1'] -= learning_rate * dW1
            self.params['b1'] -= learning_rate * db1
            self.params['W2'] -= learning_rate * dW2
            self.params['b2'] -= learning_rate * db2

            # Print loss for monitoring training progress
            if epoch % 100 == 0:
                loss = self.loss(X, y)
                print("Epoch {}: loss = {}".format(epoch, loss))


This code defines a TwoLayerNet class with an initializer that takes the input size, hidden size, and output size as arguments. The weights and biases for the two layers are initialized randomly in this function.

The forward function takes an input X and performs the forward propagation to calculate the output probabilities for each class.

The loss function calculates the cross-entropy loss between the predicted probabilities and the true labels y.

The train function performs the backpropagation to update the weights and biases based on the calculated gradients. The number of training epochs and learning rate can be specified as arguments to this function.

### Test

Here's an example of how to use the TwoLayerNet class to train and test the network on a toy dataset:

In [3]:
# Generate a toy dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])

# Initialize a neural network
net = TwoLayerNet(input_size=2, hidden_size=10, output_size=2)

# Train the neural network
net.train(X, y, num_epochs=1000)

# Test the neural network
probs = net.forward(X)
predictions = np.argmax(probs, axis=1)
print("Predictions: ", predictions)


Epoch 0: loss = 0.8791617000548932
Epoch 100: loss = 0.03272609589944909
Epoch 200: loss = 0.010130354895034843
Epoch 300: loss = 0.005517334222420798
Epoch 400: loss = 0.0036701620853277555
Epoch 500: loss = 0.002707635703438397
Epoch 600: loss = 0.0021206045443387493
Epoch 700: loss = 0.0017317523015295431
Epoch 800: loss = 0.0014568091215886065
Epoch 900: loss = 0.0012539964886349238
Predictions:  [0 1 1 0]


### Improvements 

There are several ways to improve the implementation of a two-layer neural network with softmax. Here are a few suggestions:

1. Weight initialization: The current implementation initializes the weights randomly using a Gaussian distribution. However, it is recommended to use other weight initialization methods such as Xavier or He initialization to improve convergence and avoid vanishing or exploding gradients. One possible implementation for Xavier initialization of the weights is:

In [ ]:
# Xavier initialization
self.params['W1'] = np.random.randn(input_size, hidden_size) / np.sqrt(input_size)
self.params['W2'] = np.random.randn(hidden_size, output_size) / np.sqrt(hidden_size)

2. Learning rate decay: The learning rate is a hyperparameter that determines the step size at each iteration during training. However, using a fixed learning rate may lead to suboptimal performance or slow convergence. A common technique is to gradually decrease the learning rate over time, known as learning rate decay, to fine-tune the network weights as the optimization process progresses.

In [ ]:
# Learning rate decay
learning_rate = 0.1
lr_decay = 0.95
lr_decay_epoch = 100
for epoch in range(num_epochs):
    # ...
    if epoch % lr_decay_epoch == 0:
        learning_rate *= lr_decay

3. Regularization: Overfitting can occur when the model is too complex and the training data is limited. Regularization techniques such as L1 or L2 regularization can be applied to the loss function to prevent overfitting and improve the generalization performance of the model.


In [ ]:
# L2 regularization
reg_lambda = 0.1
data_loss += 0.5 * reg_lambda * (np.sum(self.params['W1'] ** 2) + np.sum(self.params['W2'] ** 2))

4. Mini-batch training: The current implementation updates the weights using the entire training set at each iteration, which can be computationally expensive for large datasets. An alternative is to use mini-batch training, where a random subset of the training data is used at each iteration to update the weights. This can speed up the training process and improve convergence.

In [ ]:
# Mini-batch training
batch_size = 64
num_batches = len(X) // batch_size
for epoch in range(num_epochs):
    for i in range(num_batches):
        # Select a random batch of data
        batch_mask = np.random.choice(len(X), batch_size)
        X_batch = X[batch_mask]
        y_batch = y[batch_mask]

        # Forward and backward propagation using the batch data
        # ...


5. Optimization algorithm: The current implementation uses stochastic gradient descent (SGD) as the optimization algorithm. However, there are other optimization algorithms such as Adam, Adagrad, and RMSprop that can improve the convergence speed and performance of the network.

In [ ]:
# Adam optimization
beta1, beta2 = 0.9, 0.999
eps = 1e-8
mW1, vW1 = 0, 0
mW2, vW2 = 0, 0
for epoch in range(num_epochs):
    # Forward and backward propagation
    # ...
    # Update parameters using Adam optimization
    mW1 = beta1 * mW1 + (1 - beta1) * dW1
    vW1 = beta2 * vW1 + (1 - beta2) * (dW1 ** 2)
    mW2 = beta1 * mW2 + (1 - beta1) * dW2
    vW2 = beta2 * vW2 + (1 - beta2) * (dW2 ** 2)
    self.params['W1'] -= learning_rate * mW1 / (np.sqrt(vW1) + eps)
    self.params['b1'] -= learning_rate * db1
    self.params['W2'] -= learning_rate * mW2 / (np.sqrt(vW2) + eps)
    self.params['b2'] -= learning_rate * db2


### Other extensions: 


* Arbitrary activation function 
* Arbitrary loss function 
* Extension to multiple layers

In [7]:

import numpy as np

class ActivationFunction:
    def __init__(self):
        pass

    def __call__(self, x):
        raise NotImplementedError

    def derivative(self, x):
        raise NotImplementedError

class ReLU(ActivationFunction):
    def __init__(self):
        super().__init__()

    def __call__(self, x):
        return np.maximum(0, x)

    def derivative(self, x):
        return (x > 0).astype(float)

class Softmax(ActivationFunction):
    def __init__(self):
        super().__init__()

    def __call__(self, x):
        exp_scores = np.exp(x - np.max(x, axis=1, keepdims=True))
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        return probs

    def derivative(self, x):
        raise NotImplementedError

class MultiLayerNet:
    def __init__(self, input_size, hidden_sizes, output_size, activation_function, loss_function, reg_lambda=0.0):
        self.params = {}
        self.num_layers = 1 + len(hidden_sizes)
        self.layer_sizes = [input_size] + hidden_sizes + [output_size]

        for i in range(1, self.num_layers + 1):
            self.params[f'W{i}'] = np.random.randn(self.layer_sizes[i-1], self.layer_sizes[i]) / np.sqrt(self.layer_sizes[i-1])
            self.params[f'b{i}'] = np.zeros(self.layer_sizes[i])

        self.activation_function = activation_function
        self.activation_function_derivatives = [activation_function.derivative for _ in range(self.num_layers)]
        self.loss_function = loss_function
        self.reg_lambda = reg_lambda

    def forward(self, X):
        layer_output = X
        self.layer_inputs = []
        self.layer_outputs = [X]

        for i in range(1, self.num_layers + 1):
            W, b = self.params[f'W{i}'], self.params[f'b{i}']
            layer_input = np.dot(layer_output, W) + b
            self.layer_inputs.append(layer_input)
            layer_output = self.activation_function(layer_input)
            self.layer_outputs.append(layer_output)

        return layer_output

    def backward(self, X, y, output):
        delta = output - y
        dW = {}
        db = {}
        delta = delta / X.shape[0]

        for i in reversed(range(1, self.num_layers + 1)):
            layer_input = self.layer_inputs[i-1]
            activation_derivative = self.activation_function_derivatives[i-1](layer_input)
            dW[f'W{i}'] = np.dot(self.layer_outputs[i-1].T, delta) + self.reg_lambda * self.params[f'W{i}']
            db[f'b{i}'] = np.sum(delta, axis=0)
            delta = np.dot(delta, self.params[f'W{i}'].T) * activation_derivative

        return dW, db

    def loss(self, X, y, output):
        data_loss = self.loss_function(output, y)
        reg_loss = 0.0

        for i in range(1, self.num_layers + 1):
            reg_loss += 0.5 * self.reg_lambda * np.sum(self.params[f'W{i}'] ** 2)

        total_loss = data_loss + reg_loss
        return total_loss

    def train(self, X, y, num_epochs, learning_rate=0.1):
        for epoch in range(num_epochs):
            # Forward propagation
            output = self.forward(X)

            # Backward propagation
            dW, db = self.backward(X, y, output)

            # Update parameters
            for i in range(1, self.num_layers + 1):
                self.params[f'W{i}'] -= learning_rate * dW[f'W{i}']
                self.params[f'b{i}'] -= learning_rate * db[f'b{i}']

            # Print loss for monitoring training progress
            if epoch % 100 == 0:
                loss = self.loss(X, y, output)
                print(f"Epoch {epoch}, loss: {loss}")




### Test

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate a toy classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the input data
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std


# Define the mean squared error loss function
def mse_loss(output, y):
    return np.mean((output - y) ** 2)

# Create a multi-layer neural network with 2 hidden layers
net = MultiLayerNet(input_size=10, hidden_sizes=[20, 10], output_size=1,
                    activation_function=Sigmoid(), loss_function=mse_loss, reg_lambda=0.01)

# Train the network for 1000 epochs
net.train(X_train, y_train, num_epochs=1000, learning_rate=0.01)

# Evaluate the trained network on the test set
output = net.forward(X_test)
predicted_classes = np.round(output)
accuracy = np.mean(predicted_classes == y_test)
print(f"Test accuracy: {accuracy}")
